# Analisis de sentimientos con Azure IA

Partiipantes: Alexandra, Noelia, Arturo

## Descripcion



En este laboratorio obtendremos el analisis de sentimientos de los tweets de personajes representativos de la politica, en este caso aplicado a Colombia

## Primeros pasos

### 1. Obtener la API de Azure IA

## >>> Instrucciones para obtener la API

En el siguiente enlace encuentras la documentacion para obtener el codigo requerido para ejecutar este laboratorio

https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/quickstarts/client-libraries-rest-api?tabs=version-3-1&pivots=programming-language-python

## 2. Instalar paqueten necesarios para la ejecucion

In [ ]:
pip install azure-ai-textanalytics --pre

## 3. Crear variables con los datos de la API requeridos para la autenticacion

In [ ]:
key = "f50803fe315d4b51816867f0d1da98a0" 
endpoint = "https://tekstoanalizo.cognitiveservices.azure.com/"

## 4. Importar las librerias necesarias para el llamado de la API

In [ ]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

## 5. Establecer los parametros de autenticacion

In [ ]:
def autenticar_con_azure():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = autenticar_con_azure()

## 6. Establecer los parametros de lectura del dataset a analizar

In [ ]:
import csv

with open('dataset.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        datumo=row[0]
        print(f'Datos de prueba: {datumo}')

## 7. Ejecutar el analisis del dataset

## Analisis de sentimientos

In [30]:
def analisis_sentimientos(client):

    documents = [datumo]
    response = client.analyze_sentiment(documents=documents)[0]
    print("Sentimiento del documento: {}".format(response.sentiment))
    print("Puntajes generales: positivo={0:.2f}; neutral={1:.2f}; negativo={2:.2f} \n".format(
        response.confidence_scores.positive,
        response.confidence_scores.neutral,
        response.confidence_scores.negative,
    ))
    for idx, sentence in enumerate(response.sentences):
        print("Frase: {}".format(sentence.text))
        print("Frase {} sentimiento: {}".format(idx+1, sentence.sentiment))
        print("Puntuacion de la frase:\nPositivo={0:.2f}\nNeutral={1:.2f}\nNegativo={2:.2f}\n".format(
            sentence.confidence_scores.positive,
            sentence.confidence_scores.neutral,
            sentence.confidence_scores.negative,
        ))

analisis_sentimientos(client)

Sentimiento del documento: negative
Puntajes generales: positivo=0.06; neutral=0.14; negativo=0.80 

Frase: Apoyemos el derecho de soldados y policías de utilizar sus armas para defender su integridad y para defender a las personas y bienes de la acción criminal del terrorismo vandálico
Frase 1 sentimiento: negative
Puntuacion de la frase:
Positivo=0.06
Neutral=0.14
Negativo=0.80



>> Como podemos ver el primer resultado del analisis, es que el sentimiento expresado en ese tweet es **Negativo**

## Extraccion de palabras clave 

In [ ]:
def extraccion_frases_clave(client):

    try:
        documents = [datumo]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            print("\tFrases Clave:")
            for phrase in response.key_phrases:
                print("\t\t", phrase)
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Excepción encontrada. {}".format(err))
        
extraccion_frases_clave(client)

## Otras pruebas de funcionalidades

### Anaisis de sentimientos mediante mineria de opiniones

In [ ]:
def analisis_sentimientos_mineria_opiniones(client):

    documents = [datumo]

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    positive_reviews = [doc for doc in doc_result if doc.sentiment == "positivo"]
    negative_reviews = [doc for doc in doc_result if doc.sentiment == "negativo"]

    positive_mined_opinions = []
    mixed_mined_opinions = []
    negative_mined_opinions = []

    for document in doc_result:
        print("Sentimiento del documento: {}".format(document.sentiment))
        print("Puntajes generales: positivo={0:.2f}; neutral={1:.2f}; negativo={2:.2f} \n".format(
            document.confidence_scores.positive,
            document.confidence_scores.neutral,
            document.confidence_scores.negative,
        ))
        for sentence in document.sentences:
            print("Frase: {}".format(sentence.text))
            print("Sentimiento de la frase: {}".format(sentence.sentiment))
            print("Puntuacion de la frase:\nPositivo={0:.2f}\nNeutral={1:.2f}\nNegativo={2:.2f}\n".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                print("......'{}' target '{}'".format(target.sentiment, target.text))
                print("......Target score:\n......Positivo{0:.2f}\n......Negativo={1:.2f}\n".format(
                    target.confidence_scores.positive,
                    target.confidence_scores.negative,
                ))
                for assessment in mined_opinion.assessments:
                    print("......'{}' assessment '{}'".format(assessment.sentiment, assessment.text))
                    print("......Assessment score:\n......Positivo={0:.2f}\n......Negativo={1:.2f}\n".format(
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative,
                    ))
            print("\n")
        print("\n")

analisis_sentimientos_mineria_opiniones(client)

### Extraccion de entidades vinculadas al texto

In [ ]:
def Entitades_vinculadas(client):

    try:
        documents = [datumo]
        result = client.recognize_linked_entities(documents = documents)[0]

        print("Entidades vinculadas:\n")
        for entity in result.entities:
            print("\tName: ", entity.name, "\tId: ", entity.data_source_entity_id, "\tUrl: ", entity.url,
            "\n\tData Source: ", entity.data_source)
            print("\tMatches:")
            for match in entity.matches:
                print("\t\tText:", match.text)
                print("\t\tConfidence Score: {0:.2f}".format(match.confidence_score))
                print("\t\tOffset: {}".format(match.offset))
                print("\t\tLength: {}".format(match.length))
            
    except Exception as err:
        print("Excepción encontrada. {}".format(err))
Entitades_vinculadas(client)

## Referencias

- https://azure.microsoft.com/es-mx/services/cognitive-services/text-analytics/
- https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/quickstarts/client-libraries-rest-api
